.

# E11-2 Deep Learning with TensorFlow 2 and Keras

Here the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) classification task is solved by use of [Keras](https://www.tensorflow.org/guide/keras/overview) in the following steps:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

The solution require TensorFlow 2 and Keras installed in your environment.
See the [install guide](https://www.tensorflow.org/install) for details.

In [1]:
import tensorflow as tf

## Data Sets

In [2]:
# Load and prepare the datasets with Keras
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
# Normalize the integer numbers of grey color to floating-point numbers between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

## Build ANN

Build the `tf.keras.Sequential` model by stacking layers. <br>
We choose one input, one output and two hidden layers.

In [4]:
# Create a model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.<br>

__Logit__ is a vector of raw (non-normalized) predictions that a classification model generates, which is ordinarily then passed to a normalization function. If the model is solving a multi-class classification problem, logits typically become an input to the softmax function. The softmax function then generates a vector of (normalized) probabilities with one value for each possible class. __Log-odds__ serve the loss function.

In [5]:
predictions = model(x_train[:1]).numpy()
predictions


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([[ 0.09867354,  0.03404151, -0.15141569,  0.26407874,  0.36847076,
        -0.6124439 ,  0.34010354, -0.07153929,  0.19892278,  0.11951627]],
      dtype=float32)

The array predictions shows a logit for each of the ten possible outputs. The `tf.nn.softmax` function converts these logits to "probabilities" for each class. 

In [6]:
# Calculate probability
tf.nn.softmax(predictions).numpy()

array([[0.10060494, 0.09430831, 0.07834421, 0.11870086, 0.13176218,
        0.0494066 , 0.12807697, 0.08485879, 0.11121337, 0.10272383]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [7]:
#Loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.
This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

3.0076714

In [9]:
# Build the model
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [10]:
# Train the model
model.fit(x_train, y_train, epochs=20)

Train on 60000 samples
Epoch 1/20
60000/60000 [==============================] - 5s 80us/sample - loss: 0.2969 - accuracy: 0.9144
Epoch 2/20
60000/60000 [==============================] - 4s 64us/sample - loss: 0.1456 - accuracy: 0.9563
Epoch 3/20
60000/60000 [==============================] - 4s 65us/sample - loss: 0.1093 - accuracy: 0.9676
Epoch 4/20
60000/60000 [==============================] - 4s 66us/sample - loss: 0.0908 - accuracy: 0.9719
Epoch 5/20
60000/60000 [==============================] - 4s 74us/sample - loss: 0.0770 - accuracy: 0.9762
Epoch 6/20
60000/60000 [==============================] - 4s 66us/sample - loss: 0.0681 - accuracy: 0.9782
Epoch 7/20
60000/60000 [==============================] - 4s 65us/sample - loss: 0.0585 - accuracy: 0.9816
Epoch 8/20
60000/60000 [==============================] - 4s 74us/sample - loss: 0.0525 - accuracy: 0.9824
Epoch 9/20
60000/60000 [==============================] - 4s 61us/sample - loss: 0.0501 - accuracy: 0.9834
Epoch 10/20
60

The `Model.evaluate` method checks the models performance.

In [11]:
# Validate the accuracy
model.evaluate(x_test,  y_test, verbose=2)

10000/1 - 1s - loss: 0.0417 - accuracy: 0.9795


[0.0832762308759091, 0.9795]

The image classifier is now trained to ~98% accuracy on this dataset.

If you want your model to return also a probability estimation, you can wrap the trained model, and attach the softmax to it.

In [12]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: id=114420, shape=(5, 10), dtype=float32, numpy=
array([[1.2165207e-12, 2.3479211e-16, 1.8825739e-09, 1.7524793e-07,
        9.8588859e-20, 1.1241811e-13, 6.8676102e-20, 9.9999988e-01,
        1.5006782e-13, 6.2442456e-09],
       [1.8515031e-14, 1.1937815e-11, 1.0000000e+00, 1.5445101e-13,
        1.1596855e-35, 2.4955339e-12, 5.9488070e-17, 3.4441033e-23,
        8.2927826e-14, 1.6162895e-30],
       [1.1065011e-10, 9.9991632e-01, 5.9667846e-06, 7.3414746e-08,
        6.8339090e-10, 2.5935341e-09, 5.0169373e-08, 7.6368655e-05,
        1.2373980e-06, 1.7291339e-12],
       [1.0000000e+00, 1.2094549e-16, 9.3195291e-09, 2.2709813e-10,
        1.6844076e-11, 3.0318198e-08, 4.5868745e-08, 1.2504521e-11,
        2.8129617e-13, 3.7610674e-12],
       [4.9587808e-12, 1.8679913e-21, 7.3835821e-10, 6.5496649e-15,
        9.9996972e-01, 1.4489496e-14, 3.2992067e-11, 6.3828716e-09,
        1.0438394e-12, 3.0335141e-05]], dtype=float32)>

## Referrence
The TensorFlow Authors